# 🎨 風格特徵提取器 - Google Colab 教學

> **這是一個規格驅動的專案**  
> 你可以使用 `specs/style-analyzer.spec.md` 中的規格，配合 AI 工具完成開發。

## 📋 學習目標

1. 理解規格驅動開發（SDD）
2. 使用 AI 工具（Gemini）輔助開發
3. 實作文字風格分析功能
4. 體驗測試驅動開發（TDD）

## 🚀 快速開始

點擊上方的「在 Colab 中開啟」按鈕，或直接執行以下單元格。

## 步驟 1：安裝依賴套件

In [ ]:
!pip install nltk vaderSentiment pytest -q

# 下載 NLTK 資料
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

print("✅ 依賴套件安裝完成！")

## 步驟 2：查看規格文件

這個專案的完整規格在 `specs/style-analyzer.spec.md`。

**核心功能：**
1. 情感分析：計算熱情度得分 (0.0-1.0)
2. 句子統計：平均/最長/最短句子長度
3. 高頻短語：提取 TOP 3 常用短語
4. 問句比例：計算問句佔比

**驗收條件範例：**
```gherkin
Given 我有一份文字檔案 "article.txt"
  And 檔案內容至少有 500 字
When 我上傳檔案到分析器
Then 系統應該回傳 JSON 格式的分析報告
  And 報告應該包含「熱情度得分」
```

## 步驟 3：實作風格分析器

### 方法 A：使用 AI 輔助（推薦）

1. 開啟 Google AI Studio: https://aistudio.google.com/
2. 複製規格文件中的「完整規格複製區」
3. 貼給 Gemini，讓它生成程式碼
4. 將生成的程式碼貼到下方

### 方法 B：手動實作

根據規格逐步實作每個功能。

In [ ]:
# 風格分析器類別
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime
from pathlib import Path
import json
import re

class StyleAnalyzer:
    """文字風格分析器"""
    
    def __init__(self):
        self.supported_formats = ['.txt', '.md']
        self.min_length = 100
        self.sentiment_analyzer = SentimentIntensityAnalyzer()
    
    def analyze_text(self, text: str) -> dict:
        """分析文字內容"""
        # 基本統計
        word_count = len(text)
        sentences = self._split_sentences(text)
        sentence_count = len(sentences)
        
        # 情感分析
        sentiment_scores = self.sentiment_analyzer.polarity_scores(text)
        enthusiasm_score = (sentiment_scores['compound'] + 1) / 2  # 轉換為 0-1
        
        # 句子長度統計
        sentence_lengths = [len(s) for s in sentences if s.strip()]
        avg_length = sum(sentence_lengths) / len(sentence_lengths) if sentence_lengths else 0
        
        # 問句比例
        question_count = text.count('？') + text.count('?')
        question_ratio = question_count / sentence_count if sentence_count > 0 else 0
        
        # 警告
        warnings = []
        if word_count < 500:
            warnings.append(f"內容過短（{word_count}字），建議至少 500 字")
        
        return {
            "analysis_date": datetime.now().isoformat(),
            "word_count": word_count,
            "sentence_count": sentence_count,
            "metrics": {
                "enthusiasm_score": round(enthusiasm_score, 2),
                "avg_sentence_length": round(avg_length, 1),
                "max_sentence_length": max(sentence_lengths) if sentence_lengths else 0,
                "min_sentence_length": min(sentence_lengths) if sentence_lengths else 0,
                "question_ratio": round(question_ratio, 2),
                "top_phrases": []  # TODO: 實作高頻短語提取
            },
            "warnings": warnings
        }
    
    def _split_sentences(self, text: str) -> list:
        """切分句子"""
        sentences = re.split(r'[。！？\.\!\?]', text)
        return [s.strip() for s in sentences if s.strip()]

print("✅ StyleAnalyzer 類別定義完成！")

## 步驟 4：測試分析器

In [ ]:
# 建立分析器實例
analyzer = StyleAnalyzer()

# 範例文字
sample_text = """
AI 時代的軟體開發新典範

大家好！今天我想和大家分享一個令人興奮的主題：AI 如何改變軟體開發的方式。

過去，我們花大量時間在寫程式碼、除錯、測試。但現在，AI 工具如 ChatGPT、GitHub Copilot 讓開發速度提升了 10 倍！這真的太棒了！

具體來說，規格驅動開發（SDD）在 AI 時代變得更加重要。為什麼呢？因為 AI 需要清楚的指令才能產出高品質的程式碼。換句話說，你的規格寫得越清楚，AI 的產出就越準確。

讓我舉個實際案例：當我用 Cursor 開發時，我會先寫好完整的規格，然後讓 AI 根據規格生成程式碼。結果呢？測試一次就通過！這在以前是不可能的。

那麼，我們應該如何適應這個新時代？我的建議是：
1. 學習撰寫清晰的規格
2. 掌握 AI 工具的使用
3. 培養驗收和判斷的能力

總之，AI 時代的開發者不是被取代，而是被賦能。關鍵在於你是否願意學習新的協作方式。讓我們一起擁抱這個充滿可能性的未來吧！
"""

# 執行分析
result = analyzer.analyze_text(sample_text)

# 顯示結果
print("📊 分析結果：")
print(json.dumps(result, ensure_ascii=False, indent=2))

## 步驟 5：視覺化結果

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

# 設定中文字體
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'DejaVu Sans']
matplotlib.rcParams['axes.unicode_minus'] = False

# 建立圖表
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 圖表 1：熱情度得分
enthusiasm = result['metrics']['enthusiasm_score']
axes[0].barh(['Enthusiasm Score'], [enthusiasm], color='#4CAF50')
axes[0].set_xlim(0, 1)
axes[0].set_title('Enthusiasm Score', fontsize=14, fontweight='bold')
axes[0].text(enthusiasm + 0.05, 0, f'{enthusiasm:.2f}', va='center')

# 圖表 2：句子長度統計
metrics = result['metrics']
lengths = [
    metrics['avg_sentence_length'],
    metrics['max_sentence_length'],
    metrics['min_sentence_length']
]
labels = ['Avg', 'Max', 'Min']
axes[1].bar(labels, lengths, color=['#2196F3', '#FF9800', '#F44336'])
axes[1].set_title('Sentence Length', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Characters')

plt.tight_layout()
plt.show()

print(f"\n📈 字數：{result['word_count']}")
print(f"📝 句子數：{result['sentence_count']}")
print(f"❓ 問句比例：{result['metrics']['question_ratio']:.1%}")

## 步驟 6：上傳你自己的文字檔案

In [ ]:
from google.colab import files

# 上傳檔案
print("請上傳你的文字檔案（.txt 或 .md）：")
uploaded = files.upload()

# 分析上傳的檔案
for filename in uploaded.keys():
    print(f"\n分析檔案：{filename}")
    
    # 讀取檔案
    with open(filename, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 執行分析
    result = analyzer.analyze_text(content)
    
    # 顯示結果
    print(json.dumps(result, ensure_ascii=False, indent=2))

## 🎓 學習總結

### 你學到了什麼？

1. ✅ **規格驅動開發（SDD）**
   - 先定義規格（要做什麼）
   - 再實作功能（怎麼做）
   - 用測試驗證（做對了嗎）

2. ✅ **AI 輔助開發**
   - 用規格指導 AI
   - AI 生成程式碼
   - 人類驗收結果

3. ✅ **實用技能**
   - 文字分析
   - 情感分析
   - 資料視覺化

### 下一步

1. 📝 完善功能（實作高頻短語提取）
2. 🧪 增加測試（確保品質）
3. 🚀 部署應用（讓別人使用）
4. 📚 嘗試其他專案（news-to-lesson, knowledge-base）

---

**🎉 恭喜完成風格分析器教學！**

更多資源：
- [完整規格文件](../specs/style-analyzer.spec.md)
- [專案 README](../README.md)
- [SDD 學習指南](https://github.com/你的帳號/SDD-learning-guide-main)